In [1]:
import numpy as np
import pandas as pd 
%matplotlib inline
import datetime
import matplotlib.pyplot as plt
import sklearn
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from scipy.stats import zscore
from imblearn.over_sampling import SMOTE
from scipy.stats import itemfreq
from time import time

In [2]:
def createSubmission(filename, y):
    fo = open( filename , 'w' )
    fo.write("ImageId,PredictedClass\n")
    for i in range(y.shape[0]):
        fo.write(str(i)+","+str(y[i])+"\n")
    fo.close()

def nnPredict(model,x, y):
    ypredict= model.predict(x)
    score= f1_score(y , ypredict, average='macro');
    return score

In [4]:
xin= np.load("./X_train.npy")
yin= np.load("./y_train.npy")
xout= np.load("./X_test.npy")

In [5]:
#scale data
def normalize(x):
    r=zscore(x, axis=0)
    return r

In [6]:
def overSampling(x, y):
    sm = SMOTE(kind='svm')
    x_resampled, y_resampled = sm.fit_sample(x, y)
    return (x_resampled, y_resampled)

In [7]:
x= xin
y= yin
for _ in range(5):
    x,y = overSampling(x, y)
print(xin.shape)
print(x.shape)
print(itemfreq(yin))
print(itemfreq(y))

x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.3, random_state=0)
print(x_train.shape)
print(x_test.shape)

(966, 1850)
(1919, 1850)
[[  0  51]
 [  1 183]
 [  2  85]
 [  3 395]
 [  4  86]
 [  5  60]
 [  6 106]]
[[  0 208]
 [  1 183]
 [  2 394]
 [  3 395]
 [  4 247]
 [  5 221]
 [  6 271]]
(1343, 1850)
(576, 1850)


In [ ]:
#split dataset and normalization 
x_train, x_test, y_train, y_test = train_test_split(
    xin, yin, test_size=0.3, random_state=0)

x_train= normalize(x_train)
x_test= normalize(x_test)

In [ ]:
nn = MLPClassifier(hidden_layer_sizes=(1851, 400,),
                   activation= 'tanh',
                   solver= 'lbfgs',
                   alpha= 1,
                   max_iter= 500,
                   learning_rate_init= 0.001,
                   verbose= True,
                   early_stopping= True,
                   validation_fraction= 0.3,
                   #tol= 0.0001
                  )

nn.fit(x_train,y_train)
y_p= nn.predict(x_train)
print('Performance in sample:\n')
print(classification_report(y_train, y_p))
print()
print(f1_score(y_train, y_p, average= None))
print()
y_pred = nn.predict(x_test)
print("Detailed classification report:")
print()
print(classification_report(y_test, y_pred))
print()
print(f1_score(y_test, y_pred, average= None))
print()
score= nnPredict(nn, x_test, y_test)
print("F1 _macro:" + str(score))

In [ ]:
NEURAL_NETWORK_ARCH = ((1851,400,))
DATE = '10_28'

nn = MLPClassifier(hidden_layer_sizes=(1851,400,),
                   activation= 'tanh',
                   solver= 'lbfgs',
                   alpha= 1,
                   max_iter= 1000,
                   learning_rate_init= 0.001,
                   verbose= True,
                   early_stopping= True,
                   validation_fraction= 0.3,
                   tol= 0.00001
                  )
x_in= normalize(xin)
nn.fit(x_in,yin)

# Run the trained neural network on the test set and write results to a csv file
filename = './submission_nn_' + DATE + '_' + str(NEURAL_NETWORK_ARCH) + '.csv'
x_out= normalize(xout)
y_out = nn.predict(x_out)
createSubmission(filename, y_out)

In [ ]:
#Score on Kaggle achived: 0.85714 